In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import math
from datetime import datetime, timedelta  ## use timedelta to increment targ by 1 year!!!!
#from datetime import timedelta  ## use this to increment targ by 1 year!!!!
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
#import random

import sys
import os
os.chdir('C:/BankRisk')

In [2]:
dur20y = pd.DataFrame(columns=['dur','adj'])  ## used at end to save last 20y act vs naive durations of ladders
dur20y

Empty DataFrame
Columns: [dur, adj]
Index: []

In [3]:
sp610 = pd.read_csv("ccs_all365_longhist_30yr.csv",index_col=0)
sp610.index = pd.to_datetime(sp610.index)
sp610.columns = sp610.columns.astype(int)

In [4]:
sp610.tail(2)

0         1         2         3         4         5    \
Date                                                                     
2015-11-01  0.121667  0.211345  0.294967  0.376122  0.472203  0.567356   
2015-12-01  0.243333  0.357203  0.453259  0.540646  0.638303  0.731737   

                 6         7         8         9    ...        351       352  \
Date                                                ...                        
2015-11-01  0.614792  0.621198  0.625210  0.627961  ...   2.578180  2.579495   
2015-12-01  0.774439  0.775875  0.776715  0.777253  ...   2.588403  2.589692   

                 353       354       355       356       357       358  \
Date                                                                     
2015-11-01  2.580809  2.582123  2.583436  2.584749  2.586062  2.587375   
2015-12-01  2.590982  2.592271  2.593560  2.594848  2.596136  2.597424   

                 359   360  
Date                        
2015-11-01  2.588687  2.59  
2015-12-01  2.598712  2.60  

[2 rows x 361 columns]

In [5]:
#df_sp = pd.read_csv("df_spot_longhist_30y.csv",index_col=0)
df_sp = pd.read_csv("df_spot_fwd3_longhist_30y.csv",index_col=0)
df_sp['Date'] = pd.to_datetime(df_sp['Date'])
df_sp.reset_index(drop=False,inplace=True)
df_sp.set_index(['Desc','Date'],drop=True,inplace=True)
df_sp.columns = df_sp.columns.astype(int)
df_sp.tail(2)

0         1         2         3         4         5    \
Desc Date                                                                     
fwd3 2015-11-01  0.375946  0.558667  0.747881  0.851752  0.817733  0.719284   
     2015-12-01  0.540281  0.731117  0.915648  1.005605  0.956123  0.848188   

                      6         7         8         9   ...        351  \
Desc Date                                               ...              
fwd3 2015-11-01  0.651764  0.649613  0.654987  0.676612 ...   3.285509   
     2015-12-01  0.779102  0.777956  0.778837  0.788987 ...   3.282391   

                      352       353       354       355       356       357  \
Desc Date                                                                     
fwd3 2015-11-01  3.291577  3.297227  3.301063  3.303817  3.307071  3.312235   
     2015-12-01  3.288367  3.293623  3.296777  3.299455  3.302934  3.308612   

                 358  359  360  
Desc Date                       
fwd3 2015-11-01  NaN  NaN  NaN  
     2015-12-01  NaN  NaN  NaN  

[2 rows x 361 columns]

In [6]:
############################################################
############################################################
## ladder dur, MV section:

for x in [1,2,3,4,5,6,7,8,9,10,12,15,20,25,30]:
  mt = 12*x  ## swap maturity: use 1,2,3,4,5,6,7,8,9,10,12,15,20 (all * 12 to convert to months)

  ci = [x for x in np.arange(1,mt+1) if x%6==(mt%6)]  # get coupon dates for new swap
  #ci[-5:]
  t2 = pd.Series(np.zeros(mt),index=np.arange(1,mt+1))  # temp table for processing
  #t2[-5:]
  ## start sections to calc ladder durations and MVs:
  #ci[-5:]
  for i in np.arange(mt): # fill up cash flows in 1st possible ladder
    cp = sp610.ix[i,mt]/2 # get coupon
    for j in np.arange(1,mt): # move prior month cash flows up by 1 month
      t2.loc[j] = t2.loc[j+1]
    t2.loc[mt] = 0        # zero out last cash flow
    t2[ci] = t2[ci]+cp    # assign coupons to payment periods
    t2[mt] = t2[mt]+100   # add return of principal to maturity period
  # tm table for holding cfs:
  tm = pd.DataFrame(t2,columns=[sp610.index[mt-1].strftime('%Y-%m-%d')]).T
  tm.index = pd.to_datetime(tm.index)
  #tm
  for i in np.arange(mt,len(sp610)): # process another month:
    for j in np.arange(1,mt): # move prior month cash flows up by 1 month
      t2.loc[j] = t2.loc[j+1]
    t2.loc[mt] = 0        # zero out last cash flow
    cp = sp610.ix[i,mt]/2 # get coupon for new swap
    t2[ci] = t2[ci]+cp    # add new coupons to payment periods
    t2[mt] = t2[mt]+100   # add return of principal to maturity period
    d2 = pd.DataFrame(t2,columns=[sp610.index[i].strftime('%Y-%m-%d')]).T
    d2.index = pd.to_datetime(d2.index)
    tm=tm.append(d2)
  tm.to_csv(str(mt)+'mLadder_cfs_longhist2.csv')
  #str(mt)+'mLadder_cfs_longhist2.csv'

  # calc MacDurs and MVs:
  ts = list(tm.columns)
  ts = [x/12 for x in ts]
  #ts[-5:]  # list of yrs for dur calcs
  dv = pd.DataFrame(tm[[1,2,3,4]])
  dv.columns = ['mv','mvpc','dur','cpn']
  dv[['mv','mvpc','dur','cpn']] = [0,0,0,0]
  #dv.head()
  ## calc dur, mv, mv% for all ladders:
  for i in np.arange(len(tm)):
    pv= [x*y for x,y in zip(tm.ix[i][0:mt],list(df_sp.loc['df',tm.index[i]])[1:mt+1])] ## mult 2 vectors: cf*df
    pvt= [x*y for x,y in zip(pv,ts)]
    dv.ix[i]['dur'] = sum(pvt)/sum(pv)
    dv.ix[i]['mv'] = sum(pv) - 100*mt
    dv.ix[i]['mvpc'] = sum(pv) / (100*mt)
    dv.ix[i]['cpn'] = sp610.ix[dv.index[i]][mt] # pull coupon for swap maturity mt
  ## add fed funds column to first section for used in case study later:
  if (mt == 12)&('ff' not in list(dv.columns)):
    dvcol = ['mv','mvpc','dur','cpn']
    dv = dv.merge(sp610[[0]],how='left',left_index=True,right_index=True)
    dv.rename(columns={0:'ff'},inplace=True)
    dv = dv[['ff']+dvcol]
  
  #dv.tail(3)
  ## calc MV3m:  
  b3 = mt/12  # balance per month repricing
  mv3 = pd.DataFrame(sp610[3])
  mv3.rename(columns={3:'rt'},inplace=True)
  df3 = mv3.merge(df_sp.loc['df'][[1,2,3]],how='left',left_index=True,right_index=True)
  df3['cf1'] = df3['rt'].shift(2) * b3 + mt*100/3
  df3['cf2'] = df3['rt'].shift(1) * b3 + mt*100/3
  df3['cf3'] = df3['rt'] * b3 + mt*100/3
  df3['mv3'] = df3[1]*df3['cf1']+df3[2]*df3['cf2']+df3[3]*df3['cf3'] - mt*100
  #df3.head()
  dv['mv3'] = df3['mv3']
  #dv.head()
  roll3m = pd.DataFrame(pd.rolling_mean(sp610[3],3, min_periods=3))
  roll3m.columns = ['roll3m']
  sp610a = pd.rolling_mean(sp610[mt],mt, min_periods=mt)
  sp610a.dropna(inplace=True)
  #sp610a.head()
  #roll3m = pd.DataFrame(pd.rolling_mean(sp610[3],3, min_periods=3))
  #roll3m.columns = ['roll3m']
  #roll3m.head()
  dv['roll'+str(mt)+'m'] = sp610a
  dv = dv.merge(roll3m, how='left',left_index=True, right_index=True)
  #dv.tail()
  dv.to_csv('durmv_'+str(mt)+'mladder_longhist2.csv')
  #'durmv_'+str(mt)+'mladder_longhist2.csv'
  ### save last calc 20y dur vs naive dur in a table
  dur20y.loc[mt] = [ dv['dur'][-240:].mean(),  dv['dur'][-240:].mean()/(mt/12/2) ]
  dur20y.sort_index(inplace=True)
  #dur20y
  ## save copies of results according to orig maturity:
  if mt == 12:
    dv12 = dv.copy()
  elif mt == 24:
    dv24 = dv.copy()
  elif mt == 36:
    dv36 = dv.copy()
  elif mt == 48:
    dv48 = dv.copy()
  elif mt == 60:
    dv60 = dv.copy()
  elif mt == 72:
    dv72 = dv.copy()
  elif mt == 84:
    dv84 = dv.copy()
  elif mt == 96:
    dv96 = dv.copy()
  elif mt == 108:
    dv108 = dv.copy()
  elif mt == 120:
    dv120 = dv.copy()
  elif mt == 144:
    dv144 = dv.copy()
  elif mt == 180:
    dv180 = dv.copy()
  elif mt == 240:
    dv240 = dv.copy()
  elif mt == 300:
    dv300 = dv.copy()
  elif mt == 360:
    dv360 = dv.copy()


In [7]:
### STOP HERE!!! run all maturities before proceeding

In [8]:
dur20y.to_csv('durladder_last20y_adjmnts.csv')

In [9]:
#voldet.columns = pd.MultiIndex.from_product([[prn], list(voldet.columns)])
dv12.columns = pd.MultiIndex.from_product([['dv12'],list(dv12.columns)])
dv24.columns = pd.MultiIndex.from_product([['dv24'],list(dv24.columns)])
dv36.columns = pd.MultiIndex.from_product([['dv36'],list(dv36.columns)])
dv48.columns = pd.MultiIndex.from_product([['dv48'],list(dv48.columns)])
dv60.columns = pd.MultiIndex.from_product([['dv60'],list(dv60.columns)])
dv72.columns = pd.MultiIndex.from_product([['dv72'],list(dv72.columns)])
dv84.columns = pd.MultiIndex.from_product([['dv84'],list(dv84.columns)])
dv96.columns = pd.MultiIndex.from_product([['dv96'],list(dv96.columns)])
dv108.columns = pd.MultiIndex.from_product([['dv108'],list(dv108.columns)])
dv120.columns = pd.MultiIndex.from_product([['dv120'],list(dv120.columns)])
dv144.columns = pd.MultiIndex.from_product([['dv144'],list(dv144.columns)])
dv180.columns = pd.MultiIndex.from_product([['dv180'],list(dv180.columns)])
dv240.columns = pd.MultiIndex.from_product([['dv240'],list(dv240.columns)])
dv300.columns = pd.MultiIndex.from_product([['dv300'],list(dv300.columns)])
dv360.columns = pd.MultiIndex.from_product([['dv360'],list(dv360.columns)])

In [10]:
dv12.head()

dv12                                                     \
                  ff         mv      mvpc       dur       cpn       mv3   
1965-12-01  4.380000   9.872790  1.008227  0.534467  5.493205  4.784516   
1966-01-01  4.481389   9.288667  1.007741  0.534508  5.463205  4.955346   
1966-02-01  4.663889   9.148350  1.007624  0.534340  5.713205  5.366464   
1966-03-01  4.714583  10.391184  1.008659  0.534491  5.503205  5.672123   
1966-04-01  4.734861  11.174960  1.009312  0.534420  5.533205  5.637323   

                                
             roll12m    roll3m  
1965-12-01  4.854038  4.998342  
1966-01-01  4.916538  5.187601  
1966-02-01  4.990705  5.376861  
1966-03-01  5.053205  5.447833  
1966-04-01  5.117372  5.457972

In [11]:
dvtot = dv12.copy()
dvtot = dvtot.merge(dv24,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv36,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv48,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv60,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv72,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv84,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv96,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv108,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv120,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv144,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv180,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv240,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv300,how='left',left_index=True,right_index=True)
dvtot = dvtot.merge(dv360,how='left',left_index=True,right_index=True)

In [12]:
dvtot.tail()

dv12                                                          \
                  ff        mv      mvpc       dur   cpn       mv3   roll12m   
2015-08-01  0.141944  0.872831  1.000727  0.540954  0.56  0.340669  0.447500   
2015-09-01  0.141944  1.005542  1.000838  0.540972  0.53  0.372685  0.461667   
2015-10-01  0.121667  1.257880  1.001048  0.540968  0.49  0.420361  0.475833   
2015-11-01  0.121667  0.789581  1.000658  0.540824  0.64  0.394329  0.500833   
2015-12-01  0.243333  0.121900  1.000102  0.540734  0.78  0.280588  0.531667   

                          dv24              ...         dv300            \
              roll3m        mv      mvpc    ...           mv3  roll300m   
2015-08-01  0.302200  5.295329  1.002206    ...      8.516720  5.579000   
2015-09-01  0.318557  6.723434  1.002801    ...      9.317129  5.555785   
2015-10-01  0.328929  8.265452  1.003444    ...     10.509024  5.532713   
2015-11-01  0.345885  4.834748  1.002014    ...      9.858228  5.510836   
2015-12-01  0.414197  2.220284  1.000925    ...      7.014700  5.489707   

                             dv360                                        \
              roll3m            mv      mvpc        dur   cpn        mv3   
2015-08-01  0.302200  12827.186644  1.356311  10.506030  2.65  10.220064   
2015-09-01  0.318557  12797.339978  1.355482  10.483665  2.68  11.180555   
2015-10-01  0.328929  13559.200015  1.376644  10.590761  2.53  12.610828   
2015-11-01  0.345885  12924.185612  1.359005  10.562240  2.59  11.829874   
2015-12-01  0.414197  12732.078935  1.353669  10.565059  2.60   8.417641   

                                
            roll360m    roll3m  
2015-08-01  6.273397  0.302200  
2015-09-01  6.249210  0.318557  
2015-10-01  6.224912  0.328929  
2015-11-01  6.202003  0.345885  
2015-12-01  6.180567  0.414197  

[5 rows x 106 columns]

In [13]:
dvtot.to_csv('ladder_durmv_allmat_longhist.csv')

In [14]:
## methods to access multiindex data:
dvtot['dv24','mv'][10:15]  # one column

1966-10-01          NaN
1966-11-01          NaN
1966-12-01    27.638363
1967-01-01    35.049880
1967-02-01    34.315623
Name: (dv24, mv), dtype: float64

In [15]:
## methods to access multiindex cell:
dvtot.ix['2015-09-01']['dv24','dur']  # one cell  (loc works too)

1.0371136056624677

In [16]:
dur20y

dur       adj
12   0.537237  1.074474
24   1.020231  1.020231
36   1.488679  0.992452
48   1.940960  0.970480
60   2.376087  0.950435
72   2.793793  0.931264
84   3.193705  0.912487
96   3.575723  0.893931
108  3.939583  0.875463
120  4.288721  0.857744
144  4.949200  0.824867
180  5.838248  0.778433
240  7.075562  0.707556
300  8.089373  0.647150
360  8.956166  0.597078